In [1]:
import json
import os
import pandas as pd


In [2]:
# Reading in results
### bbox format: (x1, y1, x2, y2)

with open(os.path.join("./results", "data_dict.json"), "r") as f:
    res_obj = json.load(f)

In [3]:
# Reading in ground truth
### bbox format: (x1, y1, w, h)

dir_name = "far_shah_b1-b3_EOI_split"

with open(os.path.join("./", dir_name, "test/result.json"), "r") as f:
    coco_gt_obj = json.load(f)

In [4]:
res_obj[0]

{'file_name': '/home/dtron2_user/ls_dtron2_full/model/far_shah_b1-b3_EOI_split/test/images/ZF4ZoJhxQjlNSu7V-serenohotels_ss.png',
 'height': 1080,
 'width': 1920,
 'annotations': [{'iscrowd': 0,
   'bbox': [1095.8163, 970.0453, 1252.8141, 1011.1241],
   'category_id': 0,
   'bbox_mode': 0},
  {'iscrowd': 0,
   'bbox': [1095.2361, 858.6618, 1253.1652, 901.2455],
   'category_id': 0,
   'bbox_mode': 0},
  {'iscrowd': 0,
   'bbox': [861.5819, 2.8884, 1059.9102, 68.0269],
   'category_id': 0,
   'bbox_mode': 0},
  {'iscrowd': 0,
   'bbox': [1095.2179, 912.0303, 1252.0514, 956.3156],
   'category_id': 0,
   'bbox_mode': 0},
  {'iscrowd': 0,
   'bbox': [42.4671, 908.2764, 107.5965, 975.386],
   'category_id': 0,
   'bbox_mode': 0},
  {'iscrowd': 0,
   'bbox': [1212.4924, 1049.0056, 1295.7235, 1079.5737],
   'category_id': 0,
   'bbox_mode': 0},
  {'iscrowd': 0,
   'bbox': [1822.4149, 15.0026, 1911.9615, 40.7503],
   'category_id': 0,
   'bbox_mode': 0},
  {'iscrowd': 0,
   'bbox': [1467.7878

In [5]:
coco_gt_obj["annotations"][0]

{'id': 30144,
 'image_id': 2114,
 'category_id': 1,
 'segmentation': [],
 'bbox': [1872.5691699604743,
  4.268774703557313,
  41.26482213438749,
  36.996047430830046],
 'ignore': 0,
 'iscrowd': 0,
 'area': 1526.635316908565}

In [6]:
# Group By Image

imgs_gt_pred_list = {}

tot_num_pred = 0
tot_num_gt = 0

for img in coco_gt_obj["images"]:
    fn1 = img["file_name"][2:]
    img_id = img["id"]
    
    gt_list = []

    for ann1 in coco_gt_obj["annotations"]:
        if img_id == ann1["image_id"]:
            x1 = ann1["bbox"][0]
            y1 = ann1["bbox"][1]
            w = ann1["bbox"][2]
            h = ann1["bbox"][3]
            gt_list.append([x1, y1, x1 + w, y1 + h])
            tot_num_gt += 1
    
    pred_list = []
    
    for res in res_obj:
        fn2 = os.path.basename(res["file_name"])
        if fn1 == fn2:
            for ann2 in res["annotations"]:
                pred_list.append(ann2["bbox"])
                tot_num_pred += 1
    
    imgs_gt_pred_list[fn1] = [gt_list, pred_list]
            



In [7]:
keys = list(imgs_gt_pred_list.keys())


In [8]:
def calculate_iou(boxA, boxB):
    
    # Determine the coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    
    # Compute the area of intersection rectangle
    interArea = max(0, xB - xA) * max(0, yB - yA)
    
    # Compute the area of both the prediction and true bounding boxes
    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])
    
    # Compute the area of union
    unionArea = boxAArea + boxBArea - interArea
    
    # Compute the Intersection over Union by dividing the intersection area by the union area
    iou = interArea / unionArea
    
    return iou

In [9]:
def calculate_int(boxA, boxB):
    
    # Determine the coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    
    # Compute the area of intersection rectangle
    interArea = max(0, xB - xA) * max(0, yB - yA)
    
    return interArea

In [10]:
thold = .5

In [11]:
iou_scores = []

for key in keys:
    gts = imgs_gt_pred_list[key][0]
    preds = imgs_gt_pred_list[key][1]
    for i, pred in enumerate(preds):
        for j, gt in enumerate(gts):
            iou = calculate_iou(pred, gt)
            if iou > thold:
                iou_scores.append([key, i, j, iou])
                
iou_scores_df = pd.DataFrame(iou_scores, columns=["fn", "pred_num", "gt_num", "iou"])

In [12]:
iou_scores_df

,fn,pred_num,gt_num,iou
0,NSfarzKNEX0AD50v-framasoft_ss.png,2,4,0.904796
1,NSfarzKNEX0AD50v-framasoft_ss.png,3,11,0.832776
2,NSfarzKNEX0AD50v-framasoft_ss.png,4,6,0.789260
3,NSfarzKNEX0AD50v-framasoft_ss.png,5,3,0.915595
4,NSfarzKNEX0AD50v-framasoft_ss.png,6,1,0.760527
...,...,...,...,...
4905,N3HbuXBRFDRuLAMY-dollarshaveclub_ss.png,7,6,0.609785
4906,N3HbuXBRFDRuLAMY-dollarshaveclub_ss.png,8,2,0.672526
4907,N3HbuXBRFDRuLAMY-dollarshaveclub_ss.png,9,4,0.802175
4908,N3HbuXBRFDRuLAMY-dollarshaveclub_ss.png,10,1,0.638465


In [13]:
iou_scores_df_g = iou_scores_df.groupby('fn')

In [14]:
iou_full_g_dfs = []
iou_corr_g_dfs = []
iou_dang_g_pred_dfs = []
iou_dang_g_gt_dfs = []
conf_mat = []

img_ctr = 0
pred_gtr_gt_list = []
gt_gtr_pred_list = []

for name, group in iou_scores_df_g:
    
    df_subset = group[["fn", "pred_num", "gt_num", "iou"]]
    df_subset = df_subset.sort_values(by="iou", ascending=False)
    iou_full_g_dfs.append(df_subset)
    
    gt_list = list(df_subset["gt_num"])
    pred_list = list(df_subset["pred_num"])
        
    gt_set = list(set(gt_list))
    pred_set = list(set(pred_list))
    
    num_gt = len(gt_set)
    num_pred = len(pred_set)
    
    data_corr = []
    data_pred_dang = []
    data_gt_dang = []
    
    # Number of predicted > number of ground truths 
    if num_pred > num_gt:
        
        pred_gtr_gt_list.append(img_ctr)

        for i, row in df_subset.iterrows():
            if row["gt_num"] in gt_set:
                data_corr.append(row)
                gt_set.remove(row["gt_num"])
            else:
                data_pred_dang.append(row)
            
        tp = num_gt
    
    # Number of ground truths > number of predicted  
    elif num_gt > num_pred:
        
        gt_gtr_pred_list.append(img_ctr)
        
        for i, row in df_subset.iterrows():
            if row["pred_num"] in pred_set:
                data_corr.append(row)
                pred_set.remove(row["pred_num"])
            else:
                data_gt_dang.append(row)       
        
        tp = num_pred   
    
    # Number of predicted = number of ground truths
    else:
        
        for i, row in df_subset.iterrows():
            data_corr.append(row)
        
        tp = num_gt
        
    iou_corr_g_dfs.append(pd.DataFrame(data_corr))
    iou_dang_g_pred_dfs.append(pd.DataFrame(data_pred_dang))
    iou_dang_g_gt_dfs.append(pd.DataFrame(data_gt_dang))    
    
    fp = num_pred - tp
    fn = num_gt - tp
    
    conf_mat.append([tp, fp, fn])
    
    img_ctr += 1

In [15]:
tp_tot = 0
fp_tot = 0
fn_tot = 0

for img in conf_mat:
    tp_tot = tp_tot + img[0]
    fp_tot = fp_tot + img[1]
    # Would be equivalent to correctly identifying areas where no object is present... so we set it to 0 because it is not relevant
    tn_tot = 0
    fn_tot = fn_tot + img[2]
    

In [23]:
print(len(pred_gtr_gt_list))

149


In [16]:
print(pred_gtr_gt_list)
print(gt_gtr_pred_list)

[2, 3, 8, 9, 11, 13, 16, 18, 31, 40, 41, 46, 49, 50, 54, 58, 61, 64, 65, 68, 70, 77, 90, 100, 105, 109, 110, 116, 122, 127, 140, 145, 147, 148, 152, 154, 156, 160, 167, 168, 169, 170, 173, 178, 179, 181, 187, 188, 189, 192, 194, 195, 199, 200, 201, 202, 203, 206, 208, 211, 212, 215, 217, 225, 232, 234, 235, 241, 242, 244, 246, 250, 252, 256, 259, 262, 265, 269, 274, 275, 280, 286, 289, 291, 292, 298, 299, 310, 311, 313, 314, 316, 318, 321, 326, 327, 328, 330, 339, 340, 343, 344, 349, 354, 361, 366, 372, 376, 377, 380, 382, 384, 386, 395, 396, 397, 400, 407, 414, 416, 417, 425, 430, 432, 433, 438, 440, 441, 445, 448, 450, 454, 457, 461, 465, 468, 472, 473, 476, 478, 480, 482, 483, 485, 490, 491, 493, 496, 500]
[264]


In [17]:
voi = 70

In [18]:
iou_full_g_dfs[voi]

,fn,pred_num,gt_num,iou
4815,6tNdsdD6lKgDeun4-midiroll_ss.png,0,7,0.913584
4817,6tNdsdD6lKgDeun4-midiroll_ss.png,3,4,0.909403
4820,6tNdsdD6lKgDeun4-midiroll_ss.png,9,1,0.820632
4823,6tNdsdD6lKgDeun4-midiroll_ss.png,15,6,0.752306
4825,6tNdsdD6lKgDeun4-midiroll_ss.png,21,13,0.720282
4816,6tNdsdD6lKgDeun4-midiroll_ss.png,1,5,0.710415
4818,6tNdsdD6lKgDeun4-midiroll_ss.png,4,8,0.688763
4824,6tNdsdD6lKgDeun4-midiroll_ss.png,17,12,0.635686
4822,6tNdsdD6lKgDeun4-midiroll_ss.png,14,15,0.570343
4819,6tNdsdD6lKgDeun4-midiroll_ss.png,8,6,0.560629


In [19]:
iou_corr_g_dfs[voi]

,fn,pred_num,gt_num,iou
4815,6tNdsdD6lKgDeun4-midiroll_ss.png,0,7,0.913584
4817,6tNdsdD6lKgDeun4-midiroll_ss.png,3,4,0.909403
4820,6tNdsdD6lKgDeun4-midiroll_ss.png,9,1,0.820632
4823,6tNdsdD6lKgDeun4-midiroll_ss.png,15,6,0.752306
4825,6tNdsdD6lKgDeun4-midiroll_ss.png,21,13,0.720282
4816,6tNdsdD6lKgDeun4-midiroll_ss.png,1,5,0.710415
4818,6tNdsdD6lKgDeun4-midiroll_ss.png,4,8,0.688763
4824,6tNdsdD6lKgDeun4-midiroll_ss.png,17,12,0.635686
4822,6tNdsdD6lKgDeun4-midiroll_ss.png,14,15,0.570343


In [20]:
iou_dang_g_pred_dfs[voi]

,fn,pred_num,gt_num,iou
4819,6tNdsdD6lKgDeun4-midiroll_ss.png,8,6,0.560629
4821,6tNdsdD6lKgDeun4-midiroll_ss.png,10,5,0.512812


In [21]:
iou_dang_g_gt_dfs[voi]

""


In [22]:
accuracy = (tp_tot + tn_tot) / (tp_tot + fp_tot + tn_tot + fn_tot)
precision = tp_tot/(tp_tot + fp_tot)
recall = tp_tot/(tp_tot + fn_tot)
f1_score = 2 * (precision * recall) / (precision + recall)
print(accuracy)
print(precision)
print(recall)
print(f1_score)

0.9482688391038696
0.9484620085557139
0.999785269486794
0.9734476270123353
